In [27]:
import numpy as np
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget
import os
import pandas as pd
import pickle
import math
from tqdm import tqdm
from scipy.fft import fft, fftfreq
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from sklearn import svm
from sklearn.metrics import mean_absolute_error,accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,f1_score
from multiprocessing import Pool
from mpl_toolkits.mplot3d import Axes3D

In [2]:
data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')

data = data[['ax','ay', 'az', 'mx', 'my', 'mz']]
#magneto sampling rate is ~12.5 10 sample averaging
#accel sampling rate is 200 100 sample averaging
#10 sample 

In [46]:
for i in range(10):
    d = Recording(f"data/train/train_trace_00{i}.pkl")
    print(f"{d.data['ax'].samplerate} {d.data['mx'].samplerate}")


200.3029712382496 12.5189357023906
200.25856969724995 12.516160606078122
200.01380194876782 12.500862621797989
200.0496506174574 12.503103163591087
199.94755725532582 12.496722328457864
199.88475574620654 12.492797234137909
199.91096501684703 12.49443531355294
199.85738332355066 12.491086457721917
199.96265371995625 12.497665857497266
197.27620196489335 12.329762622805834


In [48]:
#downsample/filter with an average, if the input is 40 elements long and winlen is 10 output will have 4 elements
def window_average(input, winlen: int):

    numwins = int(len(input)/winlen)
    remainder = len(input)%winlen
    output=[]

    for i in range(numwins):
        output.append(np.mean(input[winlen*i:winlen*(i+1)]))
    
    return output



numtrace = 100

averageratio = int(len(ax)/len(mx))

ax = window_average(data['ax'].loc[numtrace].values, averageratio)
ay = window_average(data['ay'].loc[numtrace].values, averageratio)
az = window_average(data['az'].loc[numtrace].values, averageratio)

mx = window_average(data['mx'].loc[numtrace].values, 10)
my = window_average(data['my'].loc[numtrace].values, 10)
mz = window_average(data['mz'].loc[numtrace].values, 10)




1.5992063492063493


In [29]:
accelvec = np.array([az[0], ay[0], ax[0]])
magvec   = np.array([mz[0], my[0], mx[0]])

# proj = magvec - ((np.dot(magvec, accelvec) / np.dot(accelvec, accelvec)) * accelvec)

def getproj(a,b):
    return a - ((np.dot(a, b) / np.dot(b, b)) * b)

def getbearing(accelvec, magvec):
    proj = getproj(magvec, accelvec)
    return math.atan2(proj[1], proj[0]) * 180.0 / math.pi

print(getbearing(accelvec, magvec))


78.95337103911245


In [ ]:
#plot bearings extracted from a plot
# z,y,x
current = [0,0,0]
for point in range(5000):
    accelvec = np.array([az[point], ay[point], ax[point]])
    magvec   = np.array([mz[point], my[point], mx[point]])
    proj = getproj(magvec, accelvec)

    length = math.sqrt(proj[0]**2 + proj[1]**2)
    plt.arrow(current[0], current[1], proj[0]/length, proj[1]/length)
    current += proj

plt.plot


In [ ]:
def classify_bearings()

In [ ]:
def vecplot(vecs):
    fig = plt.figure(facecolor=plt.cm.Blues(0.2))

    ax =  fig.add_subplot(projection = '3d')
    fig.suptitle(

        f"3D-vector (2,3,4)",

        fontsize=18,

        fontweight="bold",

    )
    for vec in vecs:
        xs, ys, zs = zip(vec, (0, 0, 0))
        ax.plot(xs, ys, zs)
        ax.set_facecolor(plt.cm.Blues(0.2))

vecplot([accelvec])